a short post on using DomoLibrary to automate dataset tagging

# Project Configuration


In [68]:
import domolibrary.client.DomoAuth as dmda
import os

token_auth = dmda.DomoTokenAuth(domo_instance = 'domo-community', domo_access_token= os.environ['DOMO_DOJO_ACCESS_TOKEN'])

await token_auth.get_auth_token()

assert isinstance(token_auth.token, str)

# Get Datasets

In [69]:
import domolibrary.classes.DomoDatacenter as ddc
import pandas as pd

domo_datasets = await ddc.DomoDatacenter.search_datasets(auth=token_auth)

domo_datasets[0:5]

[DomoDataset(id='da552832-c04d-46ac-936a-f982d9d3f2e6', display_type='api', data_provider_type=None, name='SalesProject01_hello world_INT *', description=None, row_count=0, column_count=0, stream_id=None, owner={'id': '1893952720', 'name': 'Jae Wilson1', 'type': 'USER', 'group': False}, formula={}, schema=DomoDataset_Schema(dataset=..., columns=[]), tags=DomoDataset_Tags(dataset=..., tag_ls=[])),
 DomoDataset(id='a2f255fc-6c24-4e63-9b61-0c88ae7c33a5', display_type='webform', data_provider_type='webform', name='AJ | Waterfall Demo', description=None, row_count=48, column_count=4, stream_id=973, owner={'id': '1345737456', 'name': 'Aditya Jain', 'type': 'USER', 'group': False}, formula={'calculation_c4e9c63d-f4c9-4c13-a7bc-8baf83f55721': {'templateId': 2662, 'id': 'calculation_c4e9c63d-f4c9-4c13-a7bc-8baf83f55721', 'name': 'WF Values', 'formula': "SUM(CASE \n\tWHEN \n    \tDOMO_BEAST_MODE(2661) = 'Absolute' THEN `Values`\n        ELSE\n        (CASE \n\tWHEN \n    DOMO_BEAST_MODE(2658) = 

In [70]:
def process_ds_dict(ds_dict):

    proj_flow = ds_dict.get('name').split("_", 1)[0]

    # Match all digits in the string and replace them with an empty string
    proj_name = proj_flow[:-2]
    proj_phase = proj_flow[-2:]

    _ = ds_dict.get('name').split("_", 1)[1]

    ds_type = _.split("_")[-1].replace(' *', '')

    v = {
        '_id': ds_dict.get('id'),
        '_ds_name': ds_dict.get('name'),
        'provider_type': ds_dict.get('display_type') or ds_dict.get('data_provider_type'),
        'proj_name': proj_name,
        'proj_phase': proj_phase,
        'ds_type': ds_type
    }

    return v


In [71]:
def generate_tags(tag_dict):
    
    tag_ls = [ f"{key} : {tag_dict[key]} *" for key in tag_dict.keys() if not key.startswith('_')]
    
    info = {  key.replace('_', ''): tag_dict[key] for key in tag_dict.keys() if key.startswith('_')}
    
    return { 'tag_ls': tag_ls, **info }




In [72]:
import pandas as pd

domo_datasets_clean = [ process_ds_dict(domo_dataset.__dict__)for domo_dataset in domo_datasets if domo_dataset.name.endswith('*')]
pd.DataFrame(domo_datasets_clean)

,_id,_ds_name,provider_type,proj_name,proj_phase,ds_type
0,da552832-c04d-46ac-936a-f982d9d3f2e6,SalesProject01_hello world_INT *,api,SalesProject,01,INT
1,063dd227-2a2d-4563-a372-6d08c950e86d,SalesProject02_summary_by_month_DASH *,api,SalesProject,02,DASH
2,6114155e-e6a5-4c9c-bec0-5f0b8e807579,SalesProject01_hello world3_INT *,api,SalesProject,01,INT
3,e5f44d26-4028-4282-88a9-239a0f909373,SalesProject02_summary_by_day_DASH *,api,SalesProject,02,DASH


In [73]:
from pprint import pprint

update_dataset_ls = [ generate_tags(tag_dict) for tag_dict in domo_datasets_clean]
pprint(update_dataset_ls)

[{'dsname': 'SalesProject01_hello world_INT *',
  'id': 'da552832-c04d-46ac-936a-f982d9d3f2e6',
  'tag_ls': ['provider_type : api *',
             'proj_name : SalesProject *',
             'proj_phase : 01 *',
             'ds_type : INT *']},
 {'dsname': 'SalesProject02_summary_by_month_DASH *',
  'id': '063dd227-2a2d-4563-a372-6d08c950e86d',
  'tag_ls': ['provider_type : api *',
             'proj_name : SalesProject *',
             'proj_phase : 02 *',
             'ds_type : DASH *']},
 {'dsname': 'SalesProject01_hello world3_INT *',
  'id': '6114155e-e6a5-4c9c-bec0-5f0b8e807579',
  'tag_ls': ['provider_type : api *',
             'proj_name : SalesProject *',
             'proj_phase : 01 *',
             'ds_type : INT *']},
 {'dsname': 'SalesProject02_summary_by_day_DASH *',
  'id': 'e5f44d26-4028-4282-88a9-239a0f909373',
  'tag_ls': ['provider_type : api *',
             'proj_name : SalesProject *',
             'proj_phase : 02 *',
             'ds_type : DASH *']}]


In [86]:
import domolibrary.classes.DomoDataset as dmd
import asyncio

for row in update_dataset_ls:
    domo_dataset = next((domo_dataset for domo_dataset in domo_datasets if row.get('id') == domo_dataset.id))

    # remove old tags
    remove_tag_ls = [tag for tag in domo_dataset.tags.tag_ls if tag.endswith(' *')]
    
    if len(remove_tag_ls) > 0:
        domo_dataset.tags.remove(remove_tag_ls = remove_tag_ls)

    # add new tags
    tag_ls = row.get('tag_ls')
    await domo_dataset.tags.add(add_tag_ls = tag_ls)


TypeError: the JSON object must be str, bytes or bytearray, not NoneType